# Laboratório 05: Expansão de Consultas

Antes de responder as perguntas, precisamos dos documentos que serão extraídos do CSV, além das 5 consultas de um termo escolhidas na questão 2.1 do Laboratório 04 e da lista invertida obtida anteriormente.



*  **Importação das extensões necessárias** 

In [0]:
import csv
import pandas as pd
import numpy as np
import nltk
import re
import collections
import math
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('rslp')
import heapq
import time
import bisect

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.



*   **Leitura do CSV e criação do ranking**

In [0]:
colecao = pd.read_csv('https://raw.githubusercontent.com/LDVictor/ri_lab_01/master/output/results.csv')
colecao = colecao.astype(str)
#colecao = pd.read_csv('https://raw.githubusercontent.com/adautofbn/ri_lab_01/master/output/results.csv')
documentos = colecao['texto']

In [0]:
tokenizador = RegexpTokenizer(r'([A-Za-zÁáÉéÍíÓóÚúÃãÕõÇçÂâÊê]{3,27})')
stopwords = nltk.corpus.stopwords.words('portuguese') 
ranking = {}
n = 0

for texto in documentos:
  palavras = [palavra for palavra in tokenizador.tokenize(texto.lower())
           if not bool(re.search(r'\d', palavra))
           and palavra not in stopwords and len(palavra) >= 3]  
  n += 1
  for t in palavras:
    if t not in ranking.keys():
      ranking[t] = []
    ranking[t].append(n)
    
for elemento in ranking.items():
  d = dict(collections.Counter(elemento[1]))
  ranking[elemento[0]] = list(d.items())



*  **Definição das 5 consultas**

In [0]:
consultas = ["presidente", "educação", "privatização", "empresa", "bolsonaro"]

# Questão 1

**Calcule as top-10 palavras mais associadas a cada uma dessas 5 palavras de acordo com as 4 métricas que vimos na aula. Você deve produzir uma tabela similar à tabela 6.3 do capítulo 6 do livro texto (pág. 204).**


Primeiramente, iremos definir as funções para calcular cada uma das métricas: Dice's Coefficient (Dice), Mutual Information Measure (MIM), Expected Mutual Information Measure (EMIM) e Pearson's Chi-squared (χ²).

In [0]:
def valorMIM(a, b):
  na = calculaNx(a)
  nb = calculaNx(b)
  nab = calculaNab(a,b)
  mim = nab / (na * nb)
  return (mim, b)
  
def valorEMIM(a, b):
  na = calculaNx(a)
  nb = calculaNx(b)
  nab = calculaNab(a, b)
  n = calculaN()
  exp = n * (nab / (na * nb))
  if exp != 0:
    emim = nab * np.log(exp)
  else:
    emim = 0
  return (na, b)
  
def valorChiSquared(a, b):
  na = calculaNx(a)
  nb = calculaNx(b)
  nab = calculaNab(a, b)
  n = calculaN()
  chi_squared = ((nab - (1 / n) * na * nb) ** 2 / (na * nb))
  return (chi_squared, b)
  
def valorDice(a, b):
  na = calculaNx(a)
  nb = calculaNx(b)
  nab = calculaNab(a, b)
  dice = (nab / (na + nb))
  return (dice, b)

De maneira análoga, também precisamos definir como calcular os valores de Na, Nb, Nab e N para usar nas métricas definidas.

In [0]:
def calculaNx(palavraX):
  return len(ranking[palavraX])

def calculaNab(palavraA, palavraB):
  documentosA = [documento for documento, pontos in ranking[palavraA]]
  documentosB = [documento for documento, pontos in ranking[palavraB]]
  n = 0
  
  for documento in documentosA:
    if documento in documentosB:
      n += 1
      
  return n
  
def calculaN():
  return documentos.count()

Com as funções já criadas, podemos agora percorrer a lista invertida para cada consulta, armazenando os resultados dos cálculos de cada métrica. Para isso, criaremos uma função que, dada a consulta, retornará uma tabela com as 10 palavras mais associadas a cada consulta de acordo com as métricas que definimos acima.

In [0]:
def criaTabela(consulta):
  dice = []
  mim = []
  emim = []
  chi_squared = []
  
  for palavra in ranking:
    if palavra != consulta:
      bisect.insort(dice, valorDice(consulta, palavra))
      bisect.insort(mim, valorMIM(consulta, palavra))
      bisect.insort(emim, valorEMIM(consulta, palavra))
      bisect.insort(chi_squared, valorChiSquared(consulta, palavra))

  dice.reverse()    
  mim.reverse()
  emim.reverse()
  chi_squared.reverse()
  
  tabela_metricas = pd.DataFrame(columns=['MIM', 'EMIM', 'χ²', 'Dice'])

  tabela_metricas['MIM'] = [palavra[1] for palavra in mim]
  tabela_metricas['EMIM'] = [palavra[1] for palavra in emim]
  tabela_metricas['χ²'] = [palavra[1] for palavra in chi_squared]
  tabela_metricas['Dice'] = [palavra[1] for palavra in dice]
  
  tabela_metricas.index += 1
  
  return tabela_metricas.head(10)

Por fim, chamaremos a função para cada consulta do vetor.




1.   Consulta "presidente"

In [0]:
tabela_consulta_1 = criaTabela(consultas[0])
tabela_consulta_1

,MIM,EMIM,χ²,Dice
1,útil,útil,presidência,governo
2,úteis,úteis,feira,ser
3,únicos,únicos,forças,brasil
4,ótimos,único,armadas,sobre
5,ótimo,únicas,trump,país
6,ótica,única,posição,bolsonaro
7,ódios,últimos,brasil,feira
8,óculos,último,parte,ainda
9,íntima,últimas,venezuela,disse
10,índole,última,casa,segundo



2.   Consulta "educação"



In [0]:
tabela_consulta_2 = criaTabela(consultas[1])
tabela_consulta_2

,MIM,EMIM,χ²,Dice
1,íntimas,útil,constrangimento,sociais
2,íntima,úteis,substancial,instituto
3,ânima,únicos,holocausto,redes
4,âncora,único,instituto,ricardo
5,ávila,únicas,nacionalista,violência
6,álibis,única,durou,culpa
7,zumba,últimos,comentários,alunos
8,zelar,último,ricardo,constrangimento
9,zeidan,últimas,sociais,faz
10,ynaê,última,sofrimento,escolas



3.   Consulta "privatização"

In [0]:
tabela_consulta_3 = criaTabela(consultas[2])
tabela_consulta_3

,MIM,EMIM,χ²,Dice
1,ámbito,útil,rentabilidade,rentabilidade
2,zelar,úteis,juros,juros
3,véu,únicos,externo,externo
4,voos,único,melhora,melhora
5,vencidas,únicas,consistente,consistente
6,venciam,única,beneficia,beneficia
7,vastas,últimos,aprofundamento,aprofundamento
8,variações,último,impostos,impostos
9,variação,últimas,ámbito,sustentar
10,turbulências,última,zelar,pressões



4.   Consulta "empresa"

In [0]:
tabela_consulta_4 = criaTabela(consultas[3])
tabela_consulta_4

,MIM,EMIM,χ²,Dice
1,ânimo,útil,desculpas,brasília
2,ânima,úteis,irregularidades,empresas
3,zoneamento,únicos,coronel,reais
4,zimmermann,único,brasília,coronel
5,yousseff,únicas,sócio,área
6,younger,única,rita,duas
7,xingam,últimos,morre,pagamento
8,xerife,último,nicas,corrupção
9,werner,últimas,irregularidade,lima
10,wanzhou,última,delatores,risco




5.   Consulta "bolsonaro"


In [0]:
tabela_consulta_5 = criaTabela(consultas[4])
tabela_consulta_5

,MIM,EMIM,χ²,Dice
1,útil,útil,jair,brasil
2,úteis,úteis,brasil,jair
3,únicos,únicos,psl,presidente
4,ótimos,único,deputado,governo
5,íntima,únicas,campanha,ser
6,índole,única,filhos,ainda
7,ímpetos,últimos,presidência,disse
8,ênfase,último,capitão,sobre
9,étnicos,últimas,araújo,ter
10,étnicas,última,eleição,diz



**Qual métrica você acha que obteve os melhores resultados? Por que?**

Após a análise de todos os resultados das tabelas acima, a métrica do coeficiente de Dice aparentou ter os melhores resultados para as cinco consultas definidas pelo fato de ter sido a mais estável e confiável em uma variedade de palavras.

# Questão 2

**De acordo com a métrica que deu os melhores resultados na sua opinião, execute agora cada consulta (usando a abordagem documento- ou termo-por-vez)  expandido-a com: os top-3, top-5 e top-10 documentos.**

Primeiramente, precisamos da função que segue uma abordagem documento-por-vez ou termo-por-vez. Para essa questão, iremos escolher a abordagem documento-por-vez implementada na última atividade de laboratório.

In [0]:
def documentoPorVez(consulta, indices, k):
    listas_invertidas = []
    r = []
    for palavra in consulta.split(" "):
        if palavra in indices.keys():
            listas_invertidas.append(indices[palavra])
    for documento in range(1, len(documentos)+1):
        sd = 0
        for lista_invertida in listas_invertidas:
            for i in lista_invertida:
                if (i[0] == documento): 
                    sd += i[1]
                    break
        if (sd != 0):
          heapq.heappush(r, (sd, documento))
    return heapq.nlargest(k, r)

Como a métrica escolhida foi o coeficiente de Dice, faremos uso da função "valorDice" nos índices da abordagem. Sendo assim, teremos a seguinte implementação: 

In [0]:
def documentoPorVez(consulta, indices, k):
    listas_invertidas = []
    r = []
    for palavra in consulta.split(" "):
        if palavra in indices.keys():
            listas_invertidas.append(indices[palavra])
    for documento in range(1, len(documentos)+1):
        sd = 0
        for lista_invertida in listas_invertidas:
            for i in lista_invertida:
                if (i[0] == documento): 
                    sd += i[1]
                    break
        if (sd != 0):
          heapq.heappush(r, (sd, documento))
    return heapq.nlargest(k, r)

In [0]:
resultados_documento = []
k = 10

for consulta in consultas:
  resultados_documento.append(documentoPorVez(consulta, ranking, k))

**O que acontece com a precisão dos resultados em cada caso? Aumenta ou diminui? Justifique bem sua resposta.**

...